# Nystrom + Randomized SVD

In this notebook, I will developing the Nystrom method with the inclusion of the randomized SVD (rSVD) algorithm to speed up the Nystrom calculations.


### Speed Stats

**Let**:
* k = rank
* N = Size of Kernel matrix
* m = subsample

**Order**:

* Nystrom: $O(Nmk + m^3)$
* Ensemble Nystrom: $O(Nmk + N_ek^3 + C_{\mu})$
* Randomized SVD: $O(N^2k + k^3)$
* Nystrom + rSVD: $O(Nmk + k^3)$

### Algorithm

**Input**:

* K = Positive Semidefinite (PSD) Kernel matrix $\in \mathbb{R}^{NxN}$
* M = Number of subsamples (columns)
* r = rank
* P = oversampling parameter
* q = power parameter

**Output**:
* $L$

In [1]:
import numpy as np
import warnings 
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
import scipy as scio
from scipy.spatial.distance import pdist
from scipy.linalg import cho_factor, cho_solve, cholesky
from sklearn.metrics.pairwise import rbf_kernel, pairwise_kernels
from sklearn.metrics import mean_squared_error
from sklearn.utils import check_array, check_random_state
from sklearn.linear_model.ridge import _solve_cholesky_kernel as kernel_solve
from time import time
from sklearn.decomposition import TruncatedSVD
from sklearn.utils.extmath import randomized_svd
# import fbpca

%matplotlib inline
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

## Resources

#### Original Formulation



## Generate Data

In [2]:
# Import dataset
from scipy.io import loadmat


data_path = '/home/emmanuel/code/kernellib/dev/scale/nystrom/satire.mat'
data = scio.io.loadmat(data_path)['D'].toarray()
print('Size of data: {}'.format(data.shape))

n_samples = data.shape[0]
random_state = 123

Size of data: (4435, 36)


## Nystrom Approximation of a Kernel Matrix

#### Kernel Matrix of Data

In [3]:
# Linear Kernel
K = data @ data.T

print(K.shape)

(4435, 4435)


In [4]:
n_samples, d_dimensions = data.shape  # Data stats
m_subsamples = 200                    # M subsamples
n_components = 100                    # rank of matrix
random_state = 1234                   # random state for subsampling; rsvd

#### Sampling

In [15]:
# uniform sampling without replacement
rng = check_random_state(random_state)
random_indices = rng.permutation(n_samples)

# column subsample matrix
column_indices = random_indices[:m_subsamples]

# choose the columns randomly from the matrix
C = K[:, column_indices]

print('Size of the sampled K matrix, C: ', C.shape)

# get the other sampled columns
W = C[column_indices, :]

print('Size of m-by-m intersection matrix, W: ', W.shape)

Size of the sampled K matrix, C:  (4435, 200)
Size of m-by-m intersection matrix, W:  (200, 200)


### SVD

In [41]:
# %%timeit

# Perform SVD
U, D, V = np.linalg.svd(W, full_matrices=False)

# Form approximation
U_approx = np.sqrt(m_subsamples / n_samples) * C @ U @ np.diag(np.power(D, -1))
D_approx = (n_samples / m_subsamples) * np.diag(D)
# print(U_approx.shape, D_approx.shape)

# Compute approximate error
err_svd = np.linalg.norm(K - U_approx @ D_approx @ U_approx.T, 'fro')

print(f'Error: {err_svd:.2e}')

Error: 5.47e-11


### SVD + k components

In [40]:
# %%timeit

# Perform SVD
U, D, V = np.linalg.svd(W, full_matrices=False)

# Take components of matrices
U = U[:, :n_components]
D = D[:n_components]
V = V[:, :n_components]

# Form approximation
U_approx = np.sqrt(m_subsamples / n_samples) * C @ U @ np.diag(np.power(D, -1))
D_approx = (n_samples / m_subsamples) * np.diag(D)
# print(U_approx.shape, D_approx.shape)

# Compute approximate error
err_ksvd = np.linalg.norm(K - U_approx @ D_approx @ U_approx.T, 'fro')

print(f'Error: {err_ksvd:.2e}')

Error: 4.45e-11


### rSVD

In [42]:
# %%timeit 

# Perform SVD
U, D, V = randomized_svd(W, n_components=n_components)

# # Take components of matrices
# U = U[:, :n_components]
# D = D[:n_components]
# V = V[:, :n_components]

# Form approximation
U_approx = np.sqrt(m_subsamples / n_samples) * C @ U @ np.diag(np.power(D, -1))
D_approx = (n_samples / m_subsamples) * np.diag(D)
# print(U_approx.shape, D_approx.shape)

# Compute approximate error
err_rsvd = np.linalg.norm(K - U_approx @ D_approx @ U_approx.T, 'fro')

print(f'Error: {err_rsvd:.2e}')

Error: 1.62e-11


In [13]:
print('Time Experiment for normal SVD.')
%timeit np.linalg.svd(W, full_matrices=False);

# print('\nTime Experiment for randomized SVD (fb).')
# %timeit fbpca.pca(W, k=k_components, raw=True, n_iter=3);

print('\nTime Experiment for randomized SVD (scikit).')
%timeit randomized_svd(W, n_components=n_components);

Time Experiment for normal SVD.
4.11 ms ± 34.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Time Experiment for randomized SVD (scikit).
9.18 ms ± 12.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Maybe Faster

In [70]:
# Nystrom approximation of K
eps = 1e-12

# size of the data
n_samples = data.shape[0]

# choose the number of components
m_subsamples = min(n_samples, m_subsamples)

# perform rnadom uniform sampling without replacement
indices = np.random.permutation(n_samples)
basis_indices = indices[:m_subsamples]
basis = data[basis_indices]

# Construct Kernel Matrix
basis_kernel = data @ basis.T

print(basis_kernel.shape)

(4435, 200)


## Data Approximation

Input:
* $X\in \mathbb{R}^{NxN}$
* 



#### Scikit-Learn Implementation

##### Sampling

In [49]:
# Nystrom approximation of K
eps = 1e-12

# size of the data
n_samples = data.shape[0]

# choose the number of components
m_subsamples = min(n_samples, m_subsamples)

# perform rnadom uniform sampling without replacement
indices = np.random.permutation(n_samples)
basis_indices = indices[:m_subsamples]
basis = data[basis_indices]

In [50]:
print(data.shape, basis.shape)

(4435, 36) (200, 36)


##### Construct Kernel Matrices

In [55]:
# Construct Kernel Matrix
basis_kernel = basis @ basis.T

print(basis_kernel.shape)
# # construct the basis kernel
# basis_gamma = 1 / (2 * np.mean(pdist(basis, metric='euclidean')))
# basis_kernel = pairwise_kernels(basis, metric=kernel, n_jobs=n_jobs, gamma=gamma)

(200, 200)


##### SVD - Get the basis vectors

In [56]:
# get the basis vectors

# Perform the normal SVD
print('Perform Normal SVD...')
U, D, V = np.linalg.svd(basis_kernel, full_matrices=False)

# use toleraance for eigenvalues
S = np.maximum(D, eps)

Perform Normal SVD...


In [57]:
# Get normalization
normalization = np.dot(U / np.sqrt(S), V)

# get components and indices
components = basis
component_indices = basis_indices

print('Size of normalization: {}'.format(normalization.shape))
print('Size of components: {}'.format(components.shape))
print('Size of component indices: {}'.format(component_indices.shape))

Size of normalization: (200, 200)
Size of components: (200, 36)
Size of component indices: (200,)


In [63]:
L = data @ basis.T @ normalization

print(L.shape)

K_approx = L @ L.T

# Compute approximate error
err_rsvd = np.linalg.norm(K - K_approx, 'fro')

print(f'Error: {err_rsvd:.2e}')

(4435, 200)
Error: 3.67e-05


##### Randomized SVD

In [67]:
print('Perform scikit RSVD...')
U_rand, D_rand, V_rand = randomized_svd(basis_kernel, n_components=n_components)

# use toleraance for eigenvalues
S_rand = np.maximum(D_rand, eps)

normalization_r = np.dot(U_rand / np.sqrt(S_rand), V_rand)

print('Size of normalization: {}'.format(normalization_r.shape))
print('Size of components: {}'.format(components.shape))
print('Size of component indices: {}'.format(component_indices.shape))

Perform scikit RSVD...
Size of normalization: (200, 200)
Size of components: (200, 36)
Size of component indices: (200,)


#### Transform New Data

In [68]:
L = data @ basis.T @ normalization_r

print(L.shape)

K_approx = L @ L.T

# Compute approximate error
err_rsvd = np.linalg.norm(K - K_approx, 'fro')

print(f'Error: {err_rsvd:.2e}')

(4435, 200)
Error: 1.33e-05
